<a href="https://colab.research.google.com/github/jaesunkkk/22/blob/main/%E1%84%8B%E1%85%A1%E1%86%BC%E1%84%89%E1%85%A1%E1%86%BC%E1%84%87%E1%85%B3%E1%86%AF_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1-a. MNIST 데이터를 훈련, 검증, 테스트 데이터로 나눈다.

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784',version=1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(mnist.data, mnist.target, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=10000, random_state=42)

1-b. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시킨다. (n_estimators=100, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(max_iter=100, tol=20, random_state=42)
mlp_clf = MLPClassifier(random_state=42)

In [ ]:
estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train, y_train)

Training the RandomForestClassifier(random_state=42)
Training the ExtraTreesClassifier(random_state=42)
Training the LinearSVC(max_iter=100, random_state=42, tol=20)
Training the MLPClassifier(random_state=42)


In [ ]:
[estimator.score(X_val, y_val) for estimator in estimators] # 선형 SVM이 다른 분류기보다 성능이 많이 떨어진다.

[0.9692, 0.9715, 0.859, 0.9606]

1-c. 이들을 직접 투표 분류기를 사용하는 앙상블로 연결한다.

In [ ]:
from sklearn.ensemble import VotingClassifier

named_models = [
    ("RandomForest" , random_forest_clf),
    ("ExtraTree" , extra_trees_clf),
    ("LinearSVM" , svm_clf),
    ("MLP" , mlp_clf),
]

voting_clf = VotingClassifier(named_models)
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('RandomForest',
                              RandomForestClassifier(random_state=42)),
                             ('ExtraTree',
                              ExtraTreesClassifier(random_state=42)),
                             ('LinearSVM',
                              LinearSVC(max_iter=100, random_state=42, tol=20)),
                             ('MLP', MLPClassifier(random_state=42))])

In [ ]:
voting_clf.score(X_val, y_val)

0.97

1-d. 다른 분류기보다 성능이 많이 떨어졌던 선형 SVM 제거 후 VotingClassifier를 다시 평가

In [ ]:
voting_clf.set_params(LinearSVM=None)
del voting_clf.estimators_[2]

In [ ]:
voting_clf.score(X_val, y_val)

0.9737

1-d. 간접 투표 분류기를 사용하여 앙상블로 연결.

In [ ]:
voting_clf.voting = "soft"
voting_clf.score(X_val, y_val)

0.97